In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

Read YOLO data & transform

In [10]:
# DATA PATH
path = r'D:\NOTEBOOK\FYP\final_datasets_changed'
train_img_path = os.path.join(path, 'images', 'train')
train_lbl_path = os.path.join(path, 'labels', 'train')

valid_img_path = os.path.join(path, 'images', 'val')
valid_lbl_path = os.path.join(path, 'labels', 'val')

test_img_path = os.path.join(path, 'images', 'test')
test_lbl_path = os.path.join(path, 'labels', 'test')

IMG_SIZE = 128
NUM_CLASSES = 4

In [11]:
def load_yolo_data(img_dir, lbl_dir):
    images, labels = [], []
    for img_file in os.listdir(img_dir):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img_path = os.path.join(img_dir, img_file)
            label_path = os.path.join(lbl_dir, img_file.replace('.jpg', '.txt')
                                                   .replace('.jpeg', '.txt')
                                                   .replace('.png', '.txt')
                                                   .replace('.bmp', '.txt'))
            
            # Read the image
            img = cv2.imread(img_path)
            if img is None:
                print(f"Skipping: Image {img_file} does not exist or is corrupted")
                continue
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
            
            # Read all objects from the label file
            try:
                with open(label_path, 'r') as f:
                    lines = f.readlines()
            except FileNotFoundError:
                print(f"Skipping: Label file {label_path} does not exist")
                continue
            
            # Process each object (one per line)
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:
                    print(f"Warning: Label file {label_path} has an incorrect format (line content: {line})")
                    continue
                try:
                    cls_id = int(parts[0])
                    x_center = float(parts[1])
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                except ValueError:
                    print(f"Warning: Label file {label_path} contains non-numeric values (line content: {line})")
                    continue
                
                # Check validity of class ID
                if cls_id < 0 or cls_id >= NUM_CLASSES:
                    print(f"Warning: Label file {label_path} contains an invalid class ID {cls_id} (valid range: 0-{NUM_CLASSES-1})")
                    continue
                
                # Treat each object as an independent sample
                images.append(img)
                one_hot = np.zeros(NUM_CLASSES)
                one_hot[cls_id] = 1
                labels.append(one_hot)
    
    return np.array(images), np.array(labels)


In [12]:
# load data
X_train, y_train = load_yolo_data(train_img_path, train_lbl_path)
X_val, y_val = load_yolo_data(valid_img_path, valid_lbl_path)
X_test, y_test = load_yolo_data(test_img_path, test_lbl_path)

print("TRAIN:", np.bincount(y_train.argmax(axis=1)))
print("VAL:", np.bincount(y_val.argmax(axis=1)))
print("TEST:", np.bincount(y_test.argmax(axis=1)))

TRAIN: [  33 1526  103  124]
VAL: [  6 349  15  34]
TEST: [ 20 713  45  67]


CNN Model

In [13]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.01), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# train
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 292ms/step - accuracy: 0.8024 - loss: 2.2694 - val_accuracy: 0.8639 - val_loss: 0.5183
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 263ms/step - accuracy: 0.8535 - loss: 0.5765 - val_accuracy: 0.8639 - val_loss: 0.5764
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 265ms/step - accuracy: 0.8498 - loss: 0.6066 - val_accuracy: 0.8639 - val_loss: 0.5953
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 265ms/step - accuracy: 0.8573 - loss: 0.5983 - val_accuracy: 0.8639 - val_loss: 0.5491
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 269ms/step - accuracy: 0.8531 - loss: 0.5905 - val_accuracy: 0.8639 - val_loss: 0.5238
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 264ms/step - accuracy: 0.8640 - loss: 0.5425 - val_accuracy: 0.8639 - val_loss: 0.5410
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 270ms/step - accuracy: 0.8503 - loss: 0.5854 - val_accuracy: 0.8639 - val_loss: 0.5254
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 266ms/step - accuracy: 0.8658 - loss: 0.5321 - val_accu

In [15]:
# test
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"TEST ACC: {test_acc:.4f}")

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.8556 - loss: 0.5479
TEST ACC: 0.8438


In [ ]:
# pred
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # class with the greatest probability
y_true_classes = np.argmax(y_test, axis=1)  # true class

# classification_report
print(classification_report(y_true_classes, y_pred_classes, target_names=["Large Crater", "Small Crater", "Medium Crater", "Incomplete Crater"]))
# confusion_matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)


27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step
                   precision    recall  f1-score   support

     Large Crater       0.00      0.00      0.00        20
     Small Crater       0.84      1.00      0.92       713
    Medium Crater       0.00      0.00      0.00        45
Incomplete Crater       0.00      0.00      0.00        67

         accuracy                           0.84       845
        macro avg       0.21      0.25      0.23       845
     weighted avg       0.71      0.84      0.77       845

Confusion Matrix:
 [[  0  20   0   0]
 [  0 713   0   0]
 [  0  45   0   0]
 [  0  67   0   0]]


c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,